# ID3 Algorithm

In [7]:
import numpy as np
import pandas as pd
import pprint

In [8]:
tennis = pd.read_csv('./tennis.csv')

In [19]:
tennis
#type(tennis)

,Day,Outlook,Temp,Humidity,Wind,Play
0,D1,Sunny,Hot,High,Weak,No
1,D2,Sunny,Hot,High,Strong,No
2,D3,Overcast,Hot,High,Weak,Yes
3,D4,Rain,Mild,High,Weak,Yes
4,D5,Rain,Cool,Normal,Weak,Yes
5,D6,Rain,Cool,Normal,Strong,No
6,D7,Overcast,Cool,Normal,Strong,Yes
7,D8,Sunny,Mild,High,Weak,No
8,D9,Sunny,Cool,Normal,Weak,Yes
9,D10,Rain,Mild,Normal,Weak,Yes


In [11]:
#data = golf
#output_col = 'Play'
#col_name = 'Outlook'

def get_uniq_values(data, col_name) :
    vals = data[col_name].unique()
    return(vals)

In [12]:
def calculate_entropy (classes) :
    probs = classes.value_counts() / len(classes)
    ent = (-probs * np.log2(probs)).sum()
    return(ent)

In [13]:
def calculate_entropy_branch (data, colname, outputcol) :
    groups = get_uniq_values(data, colname)
    
    entropy = 0
    
    for group in groups : 
        labels = data[outputcol][data[colname] == group]
        branch_prob = len(labels) / len(data)
        branch_entropy = calculate_entropy(labels)
        entropy += branch_prob * branch_entropy
    
    return(entropy)

In [14]:
def calculate_information_gain (data, colname, outputcol) :
    root_entropy = calculate_entropy(data[outputcol])
    branch_entropy = calculate_entropy_branch(data, colname, outputcol)
    ig = root_entropy - branch_entropy
    return (ig)

In [26]:
def calculate_best_feature (data, features, outputcol) : 
    #features = data.columns.difference([outputcol])
    
    res = {}
    
    for feature in features : 
        res[feature] = calculate_information_gain(data, feature, outputcol)
        #print(res)
    
    return(res)

In [27]:
def ID3(data, originaldata, features, outputcol, parentclass):
   
    if len(data[outputcol].unique()) <= 1 : 
        return(data[outputcol].unique())
    elif len(data) == 0 : 
        return (originaldata[outputcol].mode())
    elif len(features) == 0 :
        return (parentclass)
    else :
        parentclass = data[outputcol].mode()
        
        item_values = calculate_best_feature(data, features, outputcol)
        best_feature = max(item_values, key = item_values.get)
        features.remove(best_feature)
        
        tree = {best_feature:{}}
                        
        for value in data[best_feature].unique() :
            value = value
            
            # Split the dataset along the value of the feature with the largest information gain and therwith create sub_datasets
            # sub_data = data.where(data[best_feature] == value).dropna()
            sub_data = data[data[best_feature] == value]
            #print(sub_data)
            
            #Call the ID3 algorithm for each of those sub_datasets with the new parameters --> Here the recursion comes in!
            subtree = ID3(sub_data, data, features, outputcol, parentclass)
            #print(subtree)
            
            #Add the sub tree, grown from the sub_dataset to the tree under the root node
            tree[best_feature][value] = subtree
            
            
        return(tree)   

In [28]:
def predict(query, tree, default = 'yes'):
    
    for key in list(query.keys()):
        if key in list(tree.keys()):
            try:
                result = tree[key][query[key]] 
            except:
                return default
  
            result = tree[key][query[key]]

            if isinstance(result,dict):
                return predict(query,result)
            else:
                return result
        

In [29]:
tree = ID3(tennis, tennis, ['Outlook', 'Temp', 'Humidity', 'Wind'], 'Play', 'yes')
pprint.pprint(tree)

{'Outlook': {'Overcast': array(['Yes'], dtype=object),
             'Rain': {'Wind': {'Strong': array(['No'], dtype=object),
                               'Weak': array(['Yes'], dtype=object)}},
             'Sunny': {'Humidity': {'High': array(['No'], dtype=object),
                                    'Normal': array(['Yes'], dtype=object)}}}}


In [30]:
query = {'Outlook' : 'Sunny', 'Temperature' : 'Hot', 'Humidity' : 'High'}

predict(query, tree)

array(['No'], dtype=object)